In [ ]:
import os
import glob
import re
import pyodbc
import pandas as pd
import re
import unicodedata
from collections.abc import Mapping
from typing import Dict, Optional
import textwrap


In [ ]:
CARPETA_1 = "Permisos"
CARPETA_2 = "Licencias"
NOMBRE_TABLA_ESPECIFICA = None  

In [ ]:
dfs: dict[str, pd.DataFrame] = {}

for carpeta in [CARPETA_1, CARPETA_2]:
    for db in lista_bases(carpeta):
        try:
            dfs.update(leer_access(db, NOMBRE_TABLA_ESPECIFICA))
            print(f"OK: {db}")
        except Exception as e:
            print(f"ERROR leyendo {db}: {e}")

print("\nResumen de DataFrames cargados:")
for k, df in dfs.items():
    print(f"  - {k}: {df.shape[0]} filas x {df.shape[1]} cols")


CREAR_VARIABLES = False  
if CREAR_VARIABLES:
    for k, df in dfs.items():
        globals()[k] = df
    print(f"\nSe crearon {len(dfs)} variables en el entorno con los nombres de arriba.")


Eliminacion de las bases de datos que no necesitamos 

In [ ]:
contenedor = globals().get("dfs", None)  
eliminar_df_objetivo(contenedor)

Normalizacion de el nombre de las columnas 

In [ ]:
estandarizar_columnas_inplace(dfs)

In [ ]:
for k, df in dfs.items():
    print(f"\n>>> {k}")
    display(df.head(3))  


In [ ]:
# Comparación 
comparar_columnas_y_mostrar(dfs, usar_normalizacion=False)


In [ ]:
# Mapeo para corregir la inconsistencia de 'cod_sex'
map_correccion_sexo = {'cod_sex': 'cod_sexo'}

# Aplicamos la corrección solo a los DataFrames de tipo 'licencia'
renombrar_columnas_especificas(dfs, map_correccion_sexo, tipo_df="licencia")

In [ ]:
# Mapeo para corregir la inconsistencia en vehiculo
map_correccion = {'bencina': 'bencinero' , 'ano': 'año'}

renombrar_columnas_especificas(dfs,map_correccion, tipo_df='vehiculo')

In [ ]:
analizar_diccionarios(dfs)

In [ ]:
agregar_columna_año(dfs)
print("\n✅ PASO 3: Adición de la columna 'año' completada.")


In [ ]:
# PASO 4: Aplicar los diccionarios para crear las columnas de etiquetas
aplicar_diccionarios_a_datos(dfs)
print("\n✅ PASO 4: Creación de columnas de etiquetas completada.")


In [ ]:
procesar_dataframes_vehiculos(dfs)
print("\n✅ PASO 4: Transformación de datos de vehículos completada.")

In [ ]:
dataframes_consolidados = pipeline_final_consolidacion_y_limpieza(dfs)

licencias = dataframes_consolidados.get('licencias')
vehiculos = dataframes_consolidados.get('vehiculos')

# Verificación final
if licencias is not None:
    print("\n--- Resumen DataFrame 'licencias' ---")
    licencias.info()
    display(licencias.head())

if vehiculos is not None:
    print("\n--- Resumen DataFrame 'vehiculos' ---")
    vehiculos.info()
    display(vehiculos.head())

Exportacion

In [ ]:
licencias.to_csv("licencias.csv", sep=';', encoding='utf-8-sig')

In [ ]:
vehiculos.to_csv("vehiculos.csv", sep=';', encoding='utf-8-sig')

Pipelines

In [ ]:
# === Generales ===
def normaliza_nombre(s: str) -> str:
    """Genera un nombre válido para variable/clave a partir de string de archivo o tabla."""
    s = os.path.splitext(os.path.basename(s))[0]  # quita ruta y extensión si viniera
    s = re.sub(r"\s+", "_", s.strip())
    s = re.sub(r"[^\w_]", "", s, flags=re.ASCII)
    # evita que empiece por dígito
    if re.match(r"^\d", s):
        s = f"n_{s}"
    return s

def obtener_tablas(cursor) -> list:
    """Devuelve lista de tablas 'reales' (excluye system tables y vistas si aplica)."""
    tablas = []
    for row in cursor.tables(tableType='TABLE'):
        # row.table_name, row.table_type, etc.
        # Filtra tablas del sistema si es necesario:
        if not row.table_name.startswith("MSys"):  # típicas del sistema Access
            tablas.append(row.table_name)
    return tablas

def leer_access(db_path: str, tabla_objetivo: str | None = None) -> dict[str, pd.DataFrame]:
    """
    Lee una base Access con el driver 'Microsoft Access Driver (*.mdb, *.accdb)'.
    Devuelve dict {clave: DataFrame}, donde clave = "<Archivo>__<Tabla>".
    """
    conn_str = (
        r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
        rf"DBQ={db_path};"
    )
    dfs_local = {}
    with pyodbc.connect(conn_str, autocommit=True) as conn:
        cursor = conn.cursor()

        if tabla_objetivo is None:
            tablas = obtener_tablas(cursor)
        else:
            tablas = [tabla_objetivo]

        base_name = normaliza_nombre(db_path)
        for t in tablas:
            # Lee toda la tabla
            query = f"SELECT * FROM [{t}]"
            df = pd.read_sql(query, conn)
            # Clave combinando nombre de archivo + nombre de tabla
            t_norm = normaliza_nombre(t)
            clave = f"{base_name}__{t_norm}"
            dfs_local[clave] = df

    return dfs_local

def lista_bases(carpeta: str) -> list[str]:
    """Encuentra .accdb y .mdb en la carpeta dada (no recursivo; usa ** para recursivo)."""
    patrones = [os.path.join(carpeta, "*.accdb"), os.path.join(carpeta, "*.mdb")]
    archivos = []
    for p in patrones:
        archivos.extend(glob.glob(p))
    return archivos


In [ ]:
def lista_bases(carpeta: str) -> list[str]:
    """Encuentra .accdb y .mdb en la carpeta dada (no recursivo; usa ** para recursivo)."""
    patrones = [os.path.join(carpeta, "*.accdb"), os.path.join(carpeta, "*.mdb")]
    archivos = []
    for p in patrones:
        archivos.extend(glob.glob(p))
    return archivos

In [ ]:
# ===================== Utilidades de normalización =====================

def quitar_acentos(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def split_camel(s: str) -> str:
    # Inserta espacios antes de mayúsculas para separar CamelCase
    return re.sub(r"(?<=[a-z0-9])([A-Z])", r" \1", s)

def normaliza_para_busqueda(nombre: str) -> tuple[str, str]:
    """
    Devuelve (norm_espacios, norm_junto) para búsquedas robustas.
    - norm_espacios: minus, sin acentos, separadores unificados y CamelCase separado.
    - norm_junto: como lo anterior pero sin espacios (para casos tipo VehiculosSinMotor).
    """
    s = str(nombre)
    s = quitar_acentos(s)
    s = split_camel(s)
    s = s.lower()
    # unifica separadores a espacio
    s = re.sub(r"[_\-./]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    junto = s.replace(" ", "")
    return s, junto

# ===================== Coincidencia con "vehículos sin motor" =====================

# Regex tolerante a la falta de la "i" en "vehículos": "veh[i]?culos"
PATRON_REGEX = re.compile(r"\bveh[i]?culos\s+sin\s+motor\b")

def es_objetivo(nombre: str) -> bool:
    norm, junto = normaliza_para_busqueda(nombre)
    if PATRON_REGEX.search(norm):
        return True
    # match por concatenado, con y sin la "i"
    if "vehiculossinmotor" in junto or "vehculossinmotor" in junto:
        return True
    return False

# ===================== Eliminación e impresión =====================

def eliminar_df_objetivo(contenedor_posible):
    eliminados = []
    conservados = []

    if isinstance(contenedor_posible, Mapping):
        # Modo dict (p.ej. dfs)
        claves = list(contenedor_posible.keys())
        for k in claves:
            v = contenedor_posible.get(k, None)
            if isinstance(v, pd.DataFrame) and es_objetivo(k):
                eliminados.append(k)
                del contenedor_posible[k]
            elif isinstance(v, pd.DataFrame):
                conservados.append(k)
        modo = "diccionario 'dfs'"
    else:
        # Modo variables sueltas en globals()
        vars_df = [k for k, v in globals().items() if isinstance(v, pd.DataFrame) and not k.startswith("_")]
        for k in vars_df:
            if es_objetivo(k):
                eliminados.append(k)
                del globals()[k]
            else:
                conservados.append(k)
        modo = "variables globales"

    eliminados.sort(key=str.lower)
    conservados.sort(key=str.lower)

    print(f"🔎 Modo: {modo}")
    print("🧹 Criterio: nombres que contengan 'veh(i)culos sin motor' (insensible a tildes/caso/separadores/CamelCase)\n")

    print(f"✅ Eliminados ({len(eliminados)}):")
    if eliminados:
        for n in eliminados:
            print(f"  - {n}")
    else:
        print("  (ninguno)")

    print(f"\n📦 Conservados ({len(conservados)}):")
    if conservados:
        for n in conservados:
            print(f"  - {n}")
    else:
        print("  (ninguno)")

In [ ]:
# 3) UTILIDADES DE NORMALIZACIÓN
def quitar_acentos(s: Optional[str]) -> Optional[str]:
    """Remueve diacríticos y normaliza apóstrofes."""
    if s is None:
        return s
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s.replace("’", "'")

def normaliza_nombre_col(col: str) -> str:
    """Normaliza nombres de columnas a snake_case sin tildes."""
    c = quitar_acentos(col).strip().lower()
    for ch in (" ", "-", ".", "/"):
        c = c.replace(ch, "_")
    while "__" in c:
        c = c.replace("__", "_")
    return c

def normaliza_texto_serie(ser: pd.Series) -> pd.Series:
    """Normaliza texto en Series: quita tildes, trim, lower."""
    return (ser.astype(str)
               .map(quitar_acentos)
               .str.strip()
               .str.lower())


In [ ]:
def listar_columnas_por_df(dfs: dict, ordenar_columnas: bool = False) -> str:
    """
    Genera un texto con viñetas del tipo:
      • nombre_tabla  (N columnas)
          - col1
          - col2
    Retorna el string (y también lo imprime).
    """
    lines = []
    for nombre, df in dfs.items():
        cols = list(df.columns)
        if ordenar_columnas:
            cols = sorted(cols)
        lines.append(f"  • {nombre}  ({len(cols)} columnas)")
        for c in cols:
            lines.append(f"      - {c}")
        lines.append("")  # línea en blanco entre tablas
    resultado = "\n".join(lines).rstrip()
    print(resultado)
    return resultado


In [ ]:
def estandarizar_columnas_inplace(dfs: Dict[str, pd.DataFrame]) -> None:
    """
    Modifica EN SITIO los nombres de columnas de cada DataFrame en dfs,
    usando normaliza_nombre_col. Soporta ColumnIndex y MultiIndex.
    """
    for _, df in dfs.items():
        if isinstance(df.columns, pd.MultiIndex):
            # Normaliza cada nivel de cada tupla
            new_cols = [
                tuple(normaliza_nombre_col(x) for x in tpl)
                for tpl in df.columns
            ]
            df.columns = pd.MultiIndex.from_tuples(new_cols, names=df.columns.names)
        else:
            # Normaliza columnas simples
            df.rename(columns=lambda c: normaliza_nombre_col(c), inplace=True)

In [ ]:
def es_vehiculo(clave: str) -> bool:
    parte = clave.split("__", 1)[1] if "__" in clave else clave
    t = parte.casefold().replace(" ", "").replace("_", "")
    # Robusto a tildes/variantes: "veh", "motor", "permiso(s)", "circulac(ión)"
    return (("veh" in t and "motor" in t) or
            ("permiso" in t and "circulac" in t))


In [ ]:
def clasificar_tabla(clave: str) -> str | None:
    tabla = clave.split("__", 1)[1] if "__" in clave else clave
    t = tabla.casefold().replace(" ", "").replace("_", "")
    if "diccionario" in t and "variable" in t:
        return "diccionario"
    if ("lic" in t or "licencia" in t) and "conduc" in t:
        return "licencia"
    return None


In [ ]:
def _clean_key(s: str) -> str:
    return s.casefold().replace(" ", "").replace("_", "")

def _pick_ref(items, prefer_token: str):
    """Elige referencia por substring (robusto). Si no la encuentra, usa el primero."""
    token = _clean_key(prefer_token)
    for it in items:
        if token in _clean_key(it["name"]):
            return it
    return items[0]

def comparar_columnas_y_mostrar(
    dfs: dict[str, pd.DataFrame],
    usar_normalizacion: bool = False,
    ref_licencia: str = "2019",
    ref_diccionario: str = "2019",
) -> None:
    """
    Compara columnas dentro de cada grupo ('diccionario' y 'licencia'),
    tomando como referencia la tabla que contenga `ref_diccionario` o `ref_licencia`
    (por defecto, '2019') en el nombre de la clave.
    """
    grupos = {"diccionario": [], "licencia": []}

    for k, df in dfs.items():
        tipo = clasificar_tabla(k)
        if tipo in grupos:
            cols_orig = list(map(str, df.columns))
            if usar_normalizacion:
                # Reutiliza tu normalizador; fallback simple si no existe
                try:
                    cols_cmp = set(norm_col_name(c) for c in cols_orig)
                except NameError:
                    try:
                        cols_cmp = set(normaliza_nombre_col(c) for c in cols_orig)
                    except NameError:
                        cols_cmp = set(c.strip().lower().replace(" ", "_") for c in cols_orig)
            else:
                cols_cmp = set(cols_orig)
            grupos[tipo].append({"name": k, "cols_orig": cols_orig, "cols_cmp": cols_cmp})

    for tipo, items in grupos.items():
        print(f"\n================= {tipo.upper()} =================")
        if not items:
            print("No se encontraron tablas de este tipo.")
            continue

        # elegir referencia por grupo (prioriza 2019 por defecto)
        prefer = ref_diccionario if tipo == "diccionario" else ref_licencia
        ref = _pick_ref(items, prefer)
        print(f"Referencia: {ref['name']}  ({len(ref['cols_orig'])} columnas)")

        iguales, difs = [], []
        for it in items:
            faltantes = ref["cols_cmp"] - it["cols_cmp"]
            extras    = it["cols_cmp"] - ref["cols_cmp"]
            if not faltantes and not extras:
                iguales.append(it["name"])
            else:
                difs.append((it, faltantes, extras))

        print(f"\nCoinciden con la referencia ({len(iguales)}):")
        for n in sorted(iguales):
            print(f"  - {n}")

        if not difs:
            print("\n✅ Todas las tablas de este tipo tienen las mismas columnas"
                  + (" (comparación normalizada)." if usar_normalizacion else "."))
            continue

        print(f"\n⚠️ Difieren de la referencia ({len(difs)}):")
        for it, falt, ext in difs:
            print(f"  - {it['name']}")
            if falt:
                print(f"      · Faltan vs ref: {sorted(falt)}")
            if ext:
                print(f"      · Sobran vs ref: {sorted(ext)}")

        print("\n📋 Listado de columnas por DataFrame (orden original):")
        for it in items:
            print(f"\n  • {it['name']}  ({len(it['cols_orig'])} columnas)")
            for c in it["cols_orig"]:
                print(f"      - {c}")


In [ ]:
def comparar_columnas_y_mostrar(
    dfs: dict[str, pd.DataFrame],
    usar_normalizacion: bool = False,
    # Ahora puedes añadir tokens de referencia para cada tipo
    ref_preferidos: dict[str, str] = {"licencia": "2019", "diccionario": "2019", "vehiculo": "21"}
) -> None:
    """
    Compara columnas dentro de cada grupo dinámicamente detectado.
    """
    grupos = {}  

    for k, df in dfs.items():
       
        tipo = clasificar_df_generico(k)
        if tipo:
            # Si el grupo (ej. 'vehiculo') no existe en el dict, lo crea
            if tipo not in grupos:
                grupos[tipo] = []

            cols_orig = list(map(str, df.columns))
            if usar_normalizacion:
                cols_cmp = set(normaliza_nombre_col(c) for c in cols_orig)
            else:
                cols_cmp = set(cols_orig)
            
            grupos[tipo].append({"name": k, "cols_orig": cols_orig, "cols_cmp": cols_cmp})

    for tipo, items in sorted(grupos.items()): # sorted() para un orden consistente
        print(f"\n================= {tipo.upper()} ==================")
        if not items:
            print("No se encontraron tablas de este tipo.")
            continue
        
        # Elige la referencia específica para este tipo, o usa el primer item como fallback
        prefer_token = ref_preferidos.get(tipo, "")
        ref = _pick_ref(items, prefer_token)
        print(f"Referencia: {ref['name']}  ({len(ref['cols_orig'])} columnas)")

        iguales, difs = [], []
        for it in items:
            faltantes = ref["cols_cmp"] - it["cols_cmp"]
            extras    = it["cols_cmp"] - ref["cols_cmp"]
            if not faltantes and not extras:
                iguales.append(it["name"])
            else:
                difs.append((it, faltantes, extras))

        print(f"\nCoinciden con la referencia ({len(iguales)}):")
        for n in sorted(iguales):
            print(f"  - {n}")

        if not difs:
            print(f"\n✅ Todas las tablas de este tipo tienen las mismas columnas"
                  + (" (comparación normalizada)." if usar_normalizacion else "."))
            continue

        print(f"\n⚠️ Difieren de la referencia ({len(difs)}):") 
        for it, falt, ext in difs:
            print(f"  - {it['name']}")
            if falt:
                print(f"      · Faltan vs ref: {sorted(falt)}")
            if ext:
                print(f"      · Sobran vs ref: {sorted(ext)}")

        print("\n📋 Listado de columnas por DataFrame (orden original):")
        for it in items:
            print(f"\n  • {it['name']}  ({len(it['cols_orig'])} columnas)")
            for c in it["cols_orig"]:
                print(f"      - {c}")

Renombrador 

In [ ]:
def renombrar_columnas_especificas(dfs: dict[str, pd.DataFrame], mapeo_renombre: dict[str, str], tipo_df: str):
    """
    Renombra columnas específicas para un tipo de DataFrame dado, usando la clasificación genérica.

    Args:
        dfs (dict): El diccionario principal de DataFrames.
        mapeo_renombre (dict): Un diccionario con {'nombre_antiguo': 'nombre_nuevo'}.
        tipo_df (str): El tipo de tabla a procesar (ej. 'licencia', 'diccionario', 'vehiculo').
    """
    print(f"🔄 Ejecutando renombrado específico para DataFrames tipo '{tipo_df}'...")
    modificados = 0
    for clave, df in dfs.items():
        # Usamos la función de clasificación genérica que sí reconoce a los vehículos
        if clasificar_df_generico(clave) == tipo_df:
            columnas_a_renombrar = {k: v for k, v in mapeo_renombre.items() if k in df.columns}
            
            if columnas_a_renombrar:
                df.rename(columns=columnas_a_renombrar, inplace=True)
                print(f"  - En '{clave}', se renombró: {columnas_a_renombrar}")
                modificados += 1
    
    if modificados == 0:
        print("  ✅ No se encontraron columnas para renombrar con el mapeo proporcionado.")
    else:
        print(f"  ✅ Proceso finalizado. Se modificaron {modificados} DataFrames.")

In [ ]:
def analizar_diccionarios(dfs: dict[str, pd.DataFrame], ejemplos_max: int = 5):
    """
    Analiza y muestra un resumen de cada DataFrame de tipo 'diccionario'.

    Args:
        dfs (dict): El diccionario principal de DataFrames.
        ejemplos_max (int): Número máximo de ejemplos de código->etiqueta a mostrar.
    """
    print("🔎 Analizando DataFrames de diccionarios...")
    
    # Filtramos solo los diccionarios
    dfs_diccionarios = {k: df for k, df in dfs.items() if clasificar_tabla(k) == "diccionario"}

    if not dfs_diccionarios:
        print("  No se encontraron DataFrames de tipo 'diccionario'.")
        return

    for clave, df in dfs_diccionarios.items():
        print(f"\n📘 Diccionario: {clave}")
        
        # Estandarizamos nombres de columnas para robustez
        df.rename(columns=lambda c: normaliza_nombre_col(c), inplace=True)
        
        if 'variable' not in df.columns:
            print("  ⚠️ No se encontró la columna 'variable'. Saltando este DataFrame.")
            continue
            
        variables = df['variable'].unique()
        print(f"🔢 Total de variables: {len(variables)}")
        print("\nLista de variables:")
        for var in variables:
            print(f"  - {var}")

        print("\nResumen por variable (cantidad de códigos y ejemplos):")
        for var in variables:
            sub_df = df[df['variable'] == var].dropna(subset=['valores', 'etiqueta'])
            
            # Crear ejemplos
            ejemplos = [
                f"{row['valores']} → {row['etiqueta']}"
                for _, row in sub_df.head(ejemplos_max).iterrows()
            ]
            ej_str = ";  ".join(ejemplos)
            
            # Usamos textwrap para un display limpio si la línea es muy larga
            linea_resumen = f"• {var} — {len(sub_df)} códigos numéricos"
            print(linea_resumen)
            
            wrapped_ej = textwrap.fill(ej_str, width=100, subsequent_indent='     ')
            print(f"   ej.: {wrapped_ej}")

In [ ]:
def agregar_columna_año(dfs: dict[str, pd.DataFrame]):
    """
    Agrega una columna 'año' a los DataFrames de licencias, extraído de la clave.
    """
    print("\n🔄 Agregando columna 'año' a las bases de licencias...")
    agregados = []
    sin_año = []

    for clave, df in dfs.items():
        if not isinstance(df, pd.DataFrame) or not es_base_licencia(clave):
            continue
        
        año = extraer_año(clave)
        if año is None:
            sin_año.append(clave)
            continue

        dfs[clave] = df.assign(año=pd.Series([año] * len(df), dtype="Int64"))
        agregados.append((clave, año))

    print("✅ Columna 'anio' agregada/actualizada en:")
    for clave, año in agregados:
        print(f"  - {clave}: año={año}")

    if sin_año:
        print("\n⚠️ Bases sin año detectable en el nombre:")
        for clave in sin_año:
            print(f"  - {clave}")

In [ ]:
def _preparar_mapas_desde_diccionario(df_dic: pd.DataFrame) -> dict:
    """Función auxiliar para convertir un DF de diccionario a un mapa anidado."""
    mapas = {}
    df_dic.rename(columns=lambda c: normaliza_nombre_col(c), inplace=True)
    for var_nombre in df_dic['variable'].unique():
        # Filtra, elimina duplicados y convierte a diccionario
        mapa_var = df_dic[df_dic['variable'] == var_nombre]\
                        .drop_duplicates(subset=['valores'])\
                        .set_index('valores')['etiqueta']\
                        .to_dict()
        mapas[var_nombre] = mapa_var
    return mapas

def aplicar_diccionarios_a_datos(dfs: dict[str, pd.DataFrame]):
    """
    Aplica los diccionarios a las bases de licencias para añadir columnas de etiquetas.
    """
    print("\n🔄 Aplicando diccionarios para crear columnas de etiquetas...")
    
    dfs_lic = {k: df for k, df in dfs.items() if es_base_licencia(k)}
    dfs_dic = {k: df for k, df in dfs.items() if clasificar_tabla(k) == "diccionario"}

    mapas_por_año = {}
    for clave_dic, df_dic in dfs_dic.items():
        año = extraer_año(clave_dic)
        if año:
            mapas_por_año[año] = _preparar_mapas_desde_diccionario(df_dic)

    for clave_lic, df_lic in dfs_lic.items():
        año_lic = extraer_año(clave_lic)
        print(f"\n🗂️ Año {año_lic} — Base: {clave_lic}")
        
        if not año_lic or año_lic not in mapas_por_año:
            print("  ⚠️ No se encontró un diccionario para este año.")
            continue
            
        mapas_a_usar = mapas_por_año[año_lic]
        clave_dic_usada = next(k for k in dfs_dic if str(año_lic) in k)
        print(f"📖 Diccionario usado: {clave_dic_usada}")

        columnas_creadas = []
        for col_codigo_raw, mapa in mapas_a_usar.items():
            col_codigo = normaliza_nombre_col(col_codigo_raw)
            
            if col_codigo in df_lic.columns:
                col_etiqueta = f"{col_codigo}_eti"
                
                # Forzamos la columna de datos al mismo tipo numérico que el mapa.
                df_lic[col_codigo] = pd.to_numeric(df_lic[col_codigo], errors='coerce')

                df_lic[col_etiqueta] = df_lic[col_codigo].map(mapa)
                
                mapeados = df_lic[col_etiqueta].notna().sum()
                total = len(df_lic)
                no_mapeados = total - mapeados
                
                if col_etiqueta not in columnas_creadas:
                    columnas_creadas.append(col_etiqueta)

                print(f"  • {col_etiqueta}: etiquetas={mapeados}/{total}, códigos no mapeados={no_mapeados}")

        if columnas_creadas:
            print(f"✅ Columnas de etiquetas creadas/actualizadas:\n  - " + "\n  - ".join(columnas_creadas))
        else:
            print("  ℹ️ No se crearon columnas nuevas.")

In [ ]:
def es_base_licencia(clave: str) -> bool:
    """
    Determina si un DataFrame corresponde a una base de datos de licencias (y no un diccionario).
    """
    k = clave.lower()
    # Busca 'lic'/'licencia' Y 'conduc', pero EXCLUYE 'diccionario'
    return (("lic" in k or "licencia" in k) and "conduc" in k) and ("diccionario" not in k)

def extraer_año(clave: str) -> int | None:
    """
    Extrae el año (ej. 2018, 2023) del nombre de la clave del DataFrame.
    """
    # Busca un patrón de 4 dígitos que empiece con '20'
    match = re.search(r"(20\d{2})", clave)
    return int(match.group(1)) if match else None

In [ ]:
def clasificar_df_generico(clave: str) -> str | None:
    """
    Clasifica un DataFrame en un grupo ('diccionario', 'licencia', 'vehiculo') 
    basado en su clave.
    """
    # Reutilizamos las funciones que ya tienes
    if clasificar_tabla(clave) == "diccionario":
        return "diccionario"
    if clasificar_tabla(clave) == "licencia":
        return "licencia"
    if es_vehiculo(clave):
        return "vehiculo"
    return None

In [ ]:
def procesar_dataframes_vehiculos(dfs: dict[str, pd.DataFrame], verificar: bool = True):
    """
    Suma las columnas de tipo de combustible para crear 'cantidad_vehiculos',
    verifica el total y elimina las columnas originales.

    Args:
        dfs (dict): El diccionario de DataFrames a modificar in-place.
        verificar (bool): Si es True, comprueba que la suma de combustibles coincida
                          con la suma de catalíticos antes de borrar columnas.
    """
    print("\n🔄 Procesando DataFrames de Vehículos...")
    
    # Columnas de combustible (incluye variaciones como 'bencina' y 'bencinero')
    cols_combustible_base = ["bencina", "bencinero", "diesel", "gas", "electrico", "otro"]
    
    for clave, df in dfs.items():
        if not es_vehiculo(clave):
            continue

        print(f"\n  -> Procesando '{clave}'...")
        
        # 1. Identificar las columnas de combustible que existen en este DF específico
        cols_a_sumar = [col for col in cols_combustible_base if col in df.columns]
        if not cols_a_sumar:
            print("     - ⚠️ No se encontraron columnas de combustible para sumar. Saltando.")
            continue
            
        # 2. Crear la nueva columna 'cantidad_vehiculos'
        df['cantidad_vehiculos'] = df[cols_a_sumar].sum(axis=1)
        print(f"     - ✅ Columna 'cantidad_vehiculos' creada.")

        # 3. Paso de Verificación
        cols_verificacion = ['catalitico', 'nocatalitico']
        if verificar and all(c in df.columns for c in cols_verificacion):
            suma_verificacion = df[cols_verificacion].sum(axis=1)
            
            # Comparamos si las dos sumas son iguales
            if df['cantidad_vehiculos'].equals(suma_verificacion):
                print("     - ✅ Verificación exitosa: Suma de combustibles == Suma de catalíticos.")
                
                # 4. Eliminar columnas originales solo si la verificación pasa
                df.drop(columns=cols_a_sumar + cols_verificacion, inplace=True)
                print(f"     - ✅ Columnas eliminadas: {cols_a_sumar + cols_verificacion}")
            else:
                print("     - ❌ ERROR DE VERIFICACIÓN: Las sumas no coinciden. No se eliminarán las columnas.")
        else:
            # Si no se puede verificar, simplemente se eliminan las de combustible
            print("     - ⚠️ No se pudo realizar la verificación. Eliminando solo columnas de combustible.")
            df.drop(columns=cols_a_sumar, inplace=True)
            print(f"     - ✅ Columnas eliminadas: {cols_a_sumar}")

In [ ]:
def consolidar_dataframes_por_tipo(dfs: dict[str, pd.DataFrame], tipo: str) -> pd.DataFrame | None:
    """
    Filtra los DataFrames de un tipo específico y los concatena en uno solo.

    Args:
        dfs (dict): El diccionario completo de DataFrames.
        tipo (str): El tipo a consolidar (ej. 'licencia', 'vehiculo').

    Returns:
        Un único DataFrame consolidado o None si no se encontraron DFs de ese tipo.
    """
    # 1. Filtrar los DataFrames que coinciden con el tipo deseado
    dfs_a_consolidar = {k: df for k, df in dfs.items() if clasificar_df_generico(k) == tipo}
    
    if not dfs_a_consolidar:
        print(f"ℹ️ No se encontraron DataFrames de tipo '{tipo}' para consolidar.")
        return None

    print(f" consolidating {len(dfs_a_consolidar)} DataFrames de tipo '{tipo}'...")
    
    # Extraer la lista de DataFrames para concatenar
    lista_dfs = list(dfs_a_consolidar.values())
    
    # 2. Concatenar todos los DataFrames en uno solo
    df_consolidado = pd.concat(lista_dfs, ignore_index=True)
    
    print(f"✅ DataFrame '{tipo}' consolidado exitosamente.")
    print(f"   -> Dimensiones finales: {df_consolidado.shape[0]} filas x {df_consolidado.shape[1]} columnas.")
    
    return df_consolidado


def pipeline_final_consolidacion_y_limpieza(dfs: dict[str, pd.DataFrame]) -> dict:
    """
    Ejecuta la consolidación de licencias y vehículos, y limpia el diccionario original.

    Args:
        dfs (dict): El diccionario principal de DataFrames que será modificado.

    Returns:
        Un nuevo diccionario con los DataFrames consolidados ('licencias', 'vehiculos').
    """
    print("\n🚀 INICIANDO PIPELINE FINAL: CONSOLIDACIÓN Y LIMPIEZA 🚀")
    
    # 1. Consolidar Licencias
    df_licencias = consolidar_dataframes_por_tipo(dfs, 'licencia')
    
    # 2. Consolidar Vehículos
    df_vehiculos = consolidar_dataframes_por_tipo(dfs, 'vehiculo')
    
    # 3. Limpieza del diccionario 'dfs' original
    print("\n🧹 Limpiando el diccionario 'dfs' original...")
    claves_a_eliminar = [k for k in dfs if clasificar_df_generico(k) in ['licencia', 'vehiculo', 'diccionario']]
    
    for clave in claves_a_eliminar:
        del dfs[clave]
        
    print(f"✅ Se eliminaron {len(claves_a_eliminar)} DataFrames individuales y diccionarios.")
    if not dfs:
        print("   -> El diccionario 'dfs' ahora está vacío.")
    else:
        print(f"   -> Quedan {len(dfs)} DataFrames sin clasificar en 'dfs'.")

    # 4. Crear un nuevo diccionario con los resultados finales
    dataframes_finales = {}
    if df_licencias is not None:
        dataframes_finales['licencias'] = df_licencias
    if df_vehiculos is not None:
        dataframes_finales['vehiculos'] = df_vehiculos
        
    print("\n🎉 PIPELINE FINAL COMPLETADO 🎉")
    return dataframes_finales